In [ ]:
%load_ext tensorboard

import os, sys, math
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pathlib2
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import datetime

num_classes = 2
N_Epoch = 4
N_Neuronio = 64
batch_size = 64
img_width = 850
img_height = 11


train_datagen = ImageDataGenerator(rescale=1./255)
train_ds = train_datagen.flow_from_directory(
    "./data/train_images0/train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=123
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_ds = val_datagen.flow_from_directory(
    './data/train_images0/validation',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)


class_names = list(train_ds.class_indices.keys())
print("Class names:", class_names)

for image_batch, labels_batch in train_ds:
    print("image_batch.shape = ", image_batch.shape)
    print("labels_batch.shape = ", labels_batch.shape)
    break

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Flatten(input_shape=(img_height, img_width, 1)),
    tf.keras.layers.Dense(N_Neuronio, activation='relu'),
    #tf.keras.layers.Dense(N_Neuronio, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_ds, epochs=N_Epoch, validation_data=val_ds, verbose=2, callbacks=[tensorboard_callback])

logs_base_dir = "logs/fit"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}  --host localhost
# No Powershell fazer->    del $env:TEMP\.tensorboard-info\*